In [1]:
import requests
from bs4 import BeautifulSoup
import os.path
import pandas as pd
import re
url='http://registration.baa.org/cfm_Archive/iframe_ArchiveSearch.cfm?mode=results&RequestTimeout=600&snap=47418326&'

columns=['year','bib','name','age','gender','city','state', 'country', 'overall', 'genderdiv', 'division','official',
          'net']
#193 is the country ID of US.
#Hong Kong is 82, China Mainland 41, Japan: 96, Taiwan: 185

CountryIDs=[2,4,5,7,8,10,11,12,13,14,15,16,17,23,18,19,27,20,25,21,22,24,28,205,29,30,34,31,32,26,204,99,44,36,50,37,35,39,40,185,45,46,38,43,47,42,48,49,51,52,149,201,53,54,55,56,57,58,59,71,206,61,62,63,64,210,65,66,67,70,72,73,68,75,76,77,78,69,80,81,83,82,84,90,86,85,89,87,88,91,93,94,96,95,102,97,100,101,98,208,103,107,106,105,104,108,109,110,112,115,114,118,119,120,127,125,116,123,117,113,124,128,129,130,133,132,3,139,131,136,134,138,140,141,142,143,135,144,41,145,146,147,148,151,122,153,155,156,167,168,170,157,163,169,158,159,160,162,161,174,209,164,165,154,60,166,171,173,176,175,172,177,180,179,183,184,186,187,188,181,190,191,189,178,193,192,194,195,196,197,150,200,202,203]

In [2]:
def BMlist(request_result):
    df_bm=pd.DataFrame(columns=columns)
    soup = BeautifulSoup(request_result.text,"lxml")
    maintable=soup.find_all(name='tr', attrs={'class': 'tr_header'})
    subtable = soup.find_all(name="table", attrs={"class": "table_infogrid"})
    num=len(maintable)
    if num>0:
        for i in range(num):
            (year,bibnumber,name,age,gender,city,state,country,_)=[t.text.strip() for t in maintable[i].findAll("td")]
            (overall, genderdiv, division, official, net )= [t.text.strip() for t in subtable[i].findAll("td")]
            df_bm.loc[i]=[year,bibnumber,name,age,gender,city,state,country,overall,genderdiv,division,official,net]
    return df_bm

In [13]:
def savedatatocsv(df_bm,year):
    filename=''.join(['bm',str(year),'.csv'])
    if os.path.isfile(filename):
        with open(filename,'a') as f:
            df_bm.to_csv(f,index = False,header=False,encoding='utf-8')
    else:
        df_bm.to_csv(filename,index = False,header=True,encoding='utf-8')

In [4]:
def get_with_countryID_year(year,countryID,start=1):
    param = {'StoredProcParamsOn': 'yes',
                 #'LastName': "li",
                 #'GenderID': gender,
                 "CountryOfResidenceID": countryID,
                 'VarTargetCount': 1000,
                 'records':25,
                 'start': start,
                 "RaceYearLowID": str(year),
                 'next': 'Next+25+Records'}
    r = requests.post(url, param)
    return r



In [5]:
year=2016
ID=82
requested=get_with_countryID_year(year,ID,1)
df_bm=BMlist(requested)
df_bm


,year,bib,name,age,gender,city,state,country,overall,genderdiv,division,official,net
0,2016,3604,"Ahluwalia, Aditya",31,M,Hong Kong,,HKG,5439 / 26629,4585 / 14463,2545 / 4802,3:24:02,3:22:01
1,2016,5255,"Cameron, Benjamin J.",40,M,Pok Fu Lam,,HKG,19573 / 26629,11342 / 14463,1709 / 1994,4:17:43,4:14:23
2,2016,29743,"Chan, Phebe",43,F,Hong Kong,,HKG,24507 / 26629,10923 / 12166,1782 / 1938,5:07:47,5:00:46
3,2016,19247,"Chan, Wai Yee",42,F,Hong Kong,,HKG,12489 / 26629,4077 / 12166,685 / 1938,3:47:21,3:45:28
4,2016,20091,"Chan, Hung Ki",59,M,Hong Kong,,HKG,23293 / 26629,13013 / 14463,1393 / 1563,4:51:21,4:44:49
5,2016,12584,"Cherry, Paul",51,M,Stanley,,HKG,23414 / 26629,13071 / 14463,1848 / 2032,4:49:45,4:46:18
6,2016,10341,"Cheung, Chi Hung",48,M,Kowloon,,HKG,3162 / 26629,2859 / 14463,311 / 2502,3:12:58,3:11:37
7,2016,19603,"Cheung, Pui Ling Pauline",40,F,Hong Kong,,HKG,15827 / 26629,6016 / 12166,1092 / 1938,3:58:02,3:56:29
8,2016,11018,"Chiu, Sui Wing",50,M,Hong Kong,,HKG,22033 / 26629,12419 / 14463,1770 / 2032,4:33:51,4:31:42
9,2016,24276,"Chu, Hoi Kun",63,M,Hong Kong,,HKG,21662 / 26629,12256 / 14463,757 / 993,4:28:39,4:28:36


In [19]:
def get_num_by_CountryID(year,id):
    r=get_with_countryID_year(year,id)
    strnum=re.search(r'returned [0-9]*',r.text).group() 
    #print(strnum)
    totalnumber=int(''.join(list(filter(str.isdigit, str(strnum)))))
    return totalnumber

In [7]:
print ('({})'.format([267]))

([267])


In [23]:
def get_all_data(year):
    for id in [0]:
        totalnumber=get_num_by_CountryID(year,id)
        print('ID {},Totoalnumber {}').format(id,totalnumber)
        if totalnumber>0:
            for start in range(1,totalnumber,25):
                print('{},').format(start),
                requested=get_with_countryID_year(year,id,start)
                df_bm=BMlist(requested)
                savedatatocsv(df_bm,year)

In [ ]:
get_all_data(2015)

ID 0,Totoalnumber 26675
1, 26, 51, 76, 101, 126, 151, 176, 201, 226, 251, 276, 301, 326, 351, 376, 401, 426, 451, 476, 501, 526, 551, 576, 601, 626, 651, 676, 701, 726, 751, 776, 801, 826, 851, 876, 901, 926, 951, 976, 1001, 1026, 1051, 1076, 1101, 1126, 1151, 1176, 1201, 1226, 1251, 1276, 1301, 1326, 1351, 1376, 1401, 1426, 1451, 1476, 1501, 1526, 1551, 1576, 1601, 1626, 1651, 1676, 1701, 1726, 1751, 1776, 1801, 1826, 1851, 1876, 1901, 1926, 1951, 1976, 2001, 2026, 2051, 2076, 2101, 2126, 2151, 2176, 2201, 2226, 2251, 2276, 2301, 2326, 2351, 2376, 2401, 2426, 2451, 2476, 2501, 2526, 2551, 2576, 2601, 2626, 2651, 2676, 2701, 2726, 2751, 2776, 2801, 2826, 2851, 2876, 2901, 2926, 2951, 2976, 3001, 3026, 3051, 3076, 3101, 3126, 3151, 3176, 3201, 3226, 3251, 3276, 3301, 3326, 3351, 3376, 3401, 3426, 3451, 3476, 3501, 3526, 3551, 3576, 3601, 3626, 3651, 3676, 3701, 3726, 3751, 3776, 3801, 3826, 3851, 3876, 3901, 3926, 3951, 3976, 4001, 4026, 4051, 4076, 4101, 4126, 4151, 4176, 4201, 4226, 4

In [ ]:
get_all_data(2016)

In [ ]:
get_all_data(2014)

26675